# APMC DATA TRANSFORMS

This notebook performs essential data transformations on `tblRawStateData.csv` to prepare for beta testing.
It is built from analysis first performed in the apmc_data_explorer notebook, which builds this tool, which produces
an artifact which is run through the APMC Data Explorer notebook again, which builds sections in this tool, and so on.
Eventually there should probably be splits performed for modularity, but let's cross that bridge later.

## Transformations Performed

1. **Deprecated/Sparse Field Removal**
2. **NULL/N-A String Replacement**
3. **Whitespace Normalization**
4. **Garbage Record Detection**
5. **Parent-Child Relationship Repair**
6. **SAME Keyword Record Splitting**

---

## Justification for Garbage Record Criteria

The garbage detection criteria emerge from our analysis of the WorldCovers data model and the practical realities of how the data was populated:

### 1. Childless Records (`nRawStateDataID` not used as any other record's `nRawStateDataID_parent`)

The `nRawStateDataID_parent` field implements a **self-referential hierarchy** allowing postmark variants to be grouped together (e.g., multiple size variants of the same basic design from the same town). Records that serve as parent anchors for other records have demonstrable structural importance. Conversely, records with no children that also have no content are likely scaffolding artifacts from import processes or abandoned data entry sessions.

### 2. Orphaned Records (`nRawStateDataID_parent = 0` OR `nRawStateDataID_parent = nRawStateDataID`)

Our data model analysis revealed three patterns in the parent relationship:
- **Self-referencing** (parent = self): The typical pattern for standalone catalog entries — these are "root" records that are their own parent
- **Zero/empty parent**: Records created without being assigned to a hierarchy — effectively orphaned
- **True child records** (different parent): Records that are subordinate variants of another entry

Records with zero parent or self-referencing that *also* lack content and approval represent incomplete data entry that was never finished or reviewed.

### 3. Empty `txtRawStateData`

The `txtRawStateData` field holds the **authoritative original catalog text** — the gold standard for each entry with ~97.8% population in working records. Our analysis established that "the real value resides in well-parsed descriptive text fields from original catalog sources." A record without this core content has no scholarly value and represents either a placeholder that was never filled or a corrupted import.

### 4. `approve_status != 'Approved'`

The data model implements a **contributor/reviewer workflow** with statuses: 'Approved', 'Pending', 'Rejected', 'Deleted'. Records that haven't achieved 'Approved' status haven't passed quality review. Combined with the other criteria, unapproved records with no content or structural purpose are deferred work that never materialized.

### Why ALL Criteria Must Be Met

Each criterion alone would catch too many legitimate records:
- Self-referencing is the *normal* pattern for standalone entries
- Deleted records may still be referenced by images or historical analyses
- Unapproved records may be valuable pending work

Only records meeting **all four criteria simultaneously** are truly disposable — they are empty, abandoned, unreviewed, deleted shells with no structural dependencies.

---

In [370]:
import pandas as pd
import numpy as np
import re
from pathlib import Path

# Configuration
INPUT_FILE = './wip/in/tblRawStateData_orig.csv'
OUTPUT_FILE = './wip/out/tblRawStateData.csv'
GARBAGE_FILE = './wip/out/tblRawStateData_garbage.csv'
DUPLICATES_FILE = './wip/out/tblRawStateData_duplicates.csv'

print(f"Input file: {INPUT_FILE}")
print(f"Cleaned output: {OUTPUT_FILE}")
print(f"Garbage records: {GARBAGE_FILE}")
print(f"Duplicate records: {DUPLICATES_FILE}")

Input file: ./wip/in/tblRawStateData_orig.csv
Cleaned output: ./wip/out/tblRawStateData.csv
Garbage records: ./wip/out/tblRawStateData_garbage.csv
Duplicate records: ./wip/out/tblRawStateData_duplicates.csv


## Load Data

In [371]:
# Load the CSV with all columns as strings initially to preserve data
df = pd.read_csv(INPUT_FILE, dtype=str, low_memory=False)

print(f"Loaded {len(df):,} records with {len(df.columns)} columns")
print(f"\nColumn names:\n{list(df.columns)}")

Loaded 52,046 records with 62 columns

Column names:
['nRawStateDataID', 'nRawStateDataID_parent', 'nGroupOrder', 'nStateID', 'txtRawStateData', 'txtRawStateDataTemp', 'txtWorkingData', 'txtPostmark', 'txtDatesSeen', 'txtSizes', 'txtColors', 'txtRates', 'txtRatesText', 'txtValue', 'txtTerritory', 'txtTown', 'txtTownPostmark', 'txtTownmarkShape', 'txtTownmarkLettering', 'txtTownmarkDateFormat', 'txtTownmarkFraming', 'txtTownmarkRateLocation', 'txtTownmarkRateText', 'txtTownmarkRateValue', 'txtTownmarkColor', 'nWidth', 'nHeight', 'txtOther', 'ynEarliestKnownDate', 'ynLatestKnownDate', 'nEarliestUseDay', 'txtEarliestUseMonth', 'txtEarliestUseYear', 'nEarliestUseYear', 'nLatestUseDay', 'txtLatestUseMonth', 'nLatestUseYear', 'txtLatestUseYear', 'ynManuscript', 'ynBackstamp', 'txtPublishedID', 'txtDefaultImage', 'txtPDFPage', 'ynProcessed', 'memNotes', 'ynTownNameHasExtra', 'ynManuscriptTownmarks', 'nOrder', 'ynDeleted', 'dtEntered', 'dtUpdated', 'nImageCount', 'ynForReview', 'txtReasonForRe

In [372]:
# Store original counts for reporting
original_count = len(df)
print(f"Original record count: {original_count:,}")

Original record count: 52,046


# Transformation 1: Deprecated/Sparse Field Removal

Based on a combination analysis of field population and application source code, we can remove the columns in this section

In [ ]:
# Columns to remove
COLUMNS_TO_DROP = [
    'txtPublishedID',
    'txtMarkedBy', 
    'txtPDFPage',
    'memNotes',
    'submitterId',
    'approverId',
    'dtMarkedForReview',
    'txtTownmarkRateLocation',
    'txtTownmarkRateValue',
    'txtTownmarkRateText',
    'txtReasonForReview',
    'txtTerritory',
    'ynEmailCheck',
    'txtUserEmail',
    'dtEntered',
    'dtUpdated',
    'nGroupOrder',
    'txtWorkingData',
    'ynLatestKnownDate',
    'ynEarliestKnownDate',
    'txtEarliestUseMonth',
    'txtEarliestUseYear',
    'txtLatestUseMonth',
    'txtLatestUseYear',
    'ynProcessed',
    'ynTownNameHasExtra',
    'ynManuscriptTownmarks',
    'request_status',
    'ynForReview'
]

# Verify columns exist before dropping
existing_cols_to_drop = [col for col in COLUMNS_TO_DROP if col in df.columns]
missing_cols = [col for col in COLUMNS_TO_DROP if col not in df.columns]

if missing_cols:
    print(f"Warning: These columns not found in dataframe: {missing_cols}")

df = df.drop(columns=existing_cols_to_drop)

print(f"\nRemaining columns:")
for i, col in enumerate(df.columns):
    print(f"  {i:2}. {col}")


Remaining columns:
   0. nRawStateDataID
   1. nRawStateDataID_parent
   2. nStateID
   3. txtRawStateData
   4. txtRawStateDataTemp
   5. txtPostmark
   6. txtDatesSeen
   7. txtSizes
   8. txtColors
   9. txtRates
  10. txtRatesText
  11. txtValue
  12. txtTown
  13. txtTownPostmark
  14. txtTownmarkShape
  15. txtTownmarkLettering
  16. txtTownmarkDateFormat
  17. txtTownmarkFraming
  18. txtTownmarkColor
  19. nWidth
  20. nHeight
  21. txtOther
  22. nEarliestUseDay
  23. nEarliestUseYear
  24. nLatestUseDay
  25. nLatestUseYear
  26. ynManuscript
  27. ynBackstamp
  28. txtDefaultImage
  29. nOrder
  30. ynDeleted
  31. nImageCount
  32. ynForReview
  33. approve_status


---
## Transformation 2: Replace NULL, N/A, and '--' Strings with Actual Nulls

The data contains widespread pollution from literal string values (case-insensitive) "NULL", "n/a", and "--" that should be actual null values.

In [374]:
def count_null_strings(dataframe, patterns):
    """Count occurrences of NULL-like strings across all columns."""
    counts = {}
    for col in dataframe.columns:
        col_series = dataframe[col].astype(str).str.strip().str.lower()
        for pattern in patterns:
            mask = col_series == pattern.lower()
            count = mask.sum()
            if count > 0:
                if col not in counts:
                    counts[col] = 0
                counts[col] += count
    return counts

# Patterns to replace (case-insensitive)
null_patterns = ['NULL', 'n/a', 'N/A', 'na', 'NA', '--']

# Count before replacement
before_counts = count_null_strings(df, null_patterns)
total_before = sum(before_counts.values())

print(f"NULL/N-A/-- strings found BEFORE replacement: {total_before:,}")
print(f"\nTop 10 affected columns:")
for col, count in sorted(before_counts.items(), key=lambda x: -x[1])[:10]:
    print(f"  {col}: {count:,}")

NULL/N-A/-- strings found BEFORE replacement: 1,072

Top 10 affected columns:
  txtValue: 709
  txtDatesSeen: 353
  txtRatesText: 7
  txtColors: 2
  txtSizes: 1


In [375]:
def replace_null_strings(dataframe, patterns):
    """Replace NULL-like string patterns with actual NaN (null) values."""
    df_cleaned = dataframe.copy()
    
    # Create a case-insensitive pattern
    pattern_regex = '|'.join([f'^{re.escape(p)}$' for p in patterns])
    
    for col in df_cleaned.columns:
        # Replace patterns with NaN (case-insensitive)
        df_cleaned[col] = df_cleaned[col].replace(
            to_replace=pattern_regex,
            value=np.nan,
            regex=True
        )
        # Also handle after stripping whitespace
        mask = df_cleaned[col].astype(str).str.strip().str.lower().isin([p.lower() for p in patterns])
        df_cleaned.loc[mask, col] = np.nan
    
    return df_cleaned

# Apply the replacement
df = replace_null_strings(df, null_patterns)

# Verify replacement
after_counts = count_null_strings(df, null_patterns)
total_after = sum(after_counts.values()) if after_counts else 0

print(f"NULL/N-A strings AFTER replacement: {total_after:,}")
print(f"Strings converted to null: {total_before - total_after:,}")

NULL/N-A strings AFTER replacement: 0
Strings converted to null: 1,072


---
## Transformation 2: Normalize Whitespace

Text fields may contain inconsistent whitespace that affects matching and display. This transformation:
- Removes leading and trailing whitespace
- Collapses multiple consecutive spaces into single spaces

In [376]:
def count_whitespace_issues(dataframe):
    """Count whitespace issues across string columns."""
    issues = {'leading_trailing': 0, 'multiple_inner': 0}
    
    for col in dataframe.columns:
        col_series = dataframe[col].astype(str)
        
        # Leading/trailing whitespace
        stripped = col_series.str.strip()
        issues['leading_trailing'] += (col_series != stripped).sum()
        
        # Multiple inner spaces
        has_multiple = col_series.str.contains(r'  +', regex=True, na=False)
        issues['multiple_inner'] += has_multiple.sum()
    
    return issues

ws_before = count_whitespace_issues(df)
print(f"Whitespace issues BEFORE normalization:")
print(f"  Leading/trailing whitespace: {ws_before['leading_trailing']:,}")
print(f"  Multiple inner spaces: {ws_before['multiple_inner']:,}")

Whitespace issues BEFORE normalization:
  Leading/trailing whitespace: 87,529
  Multiple inner spaces: 254


In [377]:
def normalize_whitespace(dataframe):
    """Normalize whitespace in all string columns."""
    df_cleaned = dataframe.copy()
    
    for col in df_cleaned.columns:
        if df_cleaned[col].dtype == object:
            # Convert to string first
            df_cleaned[col] = df_cleaned[col].astype(str)
            # Replace newlines/carriage returns with spaces  <-- ADD THIS
            df_cleaned[col] = df_cleaned[col].str.replace(r'[\r\n]+', ' ', regex=True)
            # Strip leading/trailing whitespace
            df_cleaned[col] = df_cleaned[col].str.strip()
            # Collapse multiple spaces into single space
            df_cleaned[col] = df_cleaned[col].str.replace(r'  +', ' ', regex=True)
            # Restore NaN for 'nan' strings created by astype
            df_cleaned.loc[df_cleaned[col] == 'nan', col] = np.nan
    
    return df_cleaned

df = normalize_whitespace(df)

ws_after = count_whitespace_issues(df)
print(f"Whitespace issues AFTER normalization:")
print(f"  Leading/trailing whitespace: {ws_after['leading_trailing']:,}")
print(f"  Multiple inner spaces: {ws_after['multiple_inner']:,}")

Whitespace issues AFTER normalization:
  Leading/trailing whitespace: 0
  Multiple inner spaces: 0


---
## Transformation 3: Garbage Record Detection

Identify records that meet ALL of the following criteria (indicating they are disposable):
1. Childless (not referenced as parent by any other record)
2. Orphaned (parent is 0, empty, or self-referencing)
3. Empty content (txtRawStateData is null/empty)
4. Not approved

Note that I used to remove all records marked with ynDelete as 1, but somehow some legitimate records had been tagged, so I've removed that.

In [378]:
# Find all IDs that are used as parents
used_as_parent = set(df['nRawStateDataID_parent'].dropna().astype(str).unique())
print(f"Unique parent IDs referenced: {len(used_as_parent):,}")

# Criterion 1: Childless (not used as parent by other records)
df['_is_childless'] = ~df['nRawStateDataID'].astype(str).isin(used_as_parent)
print(f"1. Childless records: {df['_is_childless'].sum():,}")

# Criterion 2: Orphaned (parent is 0, empty, or self)
def is_orphaned(row):
    parent = str(row['nRawStateDataID_parent']) if pd.notna(row['nRawStateDataID_parent']) else ''
    self_id = str(row['nRawStateDataID']) if pd.notna(row['nRawStateDataID']) else ''
    return parent == '' or parent == '0' or parent == self_id

df['_is_orphaned'] = df.apply(is_orphaned, axis=1)
print(f"2. Orphaned records (parent=0/empty/self): {df['_is_orphaned'].sum():,}")

# Criterion 3: Empty txtRawStateData
df['_empty_content'] = df['txtRawStateData'].isna() | (df['txtRawStateData'].astype(str).str.strip() == '')
print(f"3. Empty txtRawStateData: {df['_empty_content'].sum():,}")

# Criterion 4: Not Approved
df['_not_approved'] = df['approve_status'] != 'Approved'
print(f"4. Not Approved: {df['_not_approved'].sum():,}")

Unique parent IDs referenced: 43,055
1. Childless records: 8,992
2. Orphaned records (parent=0/empty/self): 51,842
3. Empty txtRawStateData: 1,119
4. Not Approved: 535


In [379]:
# Combine all criteria - a record is garbage only if ALL conditions are true
df['_is_garbage'] = (
    df['_is_childless'] &
    df['_is_orphaned'] &
    df['_empty_content'] &
    df['_not_approved']
)

garbage_count = df['_is_garbage'].sum()
clean_count = len(df) - garbage_count

print(f"\n=== GARBAGE DETECTION RESULTS ===")
print(f"Total records: {len(df):,}")
print(f"Garbage records (ALL criteria met): {garbage_count:,} ({garbage_count/len(df)*100:.2f}%)")
print(f"Clean records: {clean_count:,} ({clean_count/len(df)*100:.2f}%)")


=== GARBAGE DETECTION RESULTS ===
Total records: 52,046
Garbage records (ALL criteria met): 335 (0.64%)
Clean records: 51,711 (99.36%)


In [380]:
# Show sample garbage records for verification
if garbage_count > 0:
    print("\n=== Sample Garbage Records (first 10) ===")
    garbage_df = df[df['_is_garbage']]
    display_cols = ['nRawStateDataID', 'nRawStateDataID_parent', 'txtRawStateData', 
                    'txtTown', 'approve_status', 'ynDeleted']
    display_cols = [c for c in display_cols if c in garbage_df.columns]
    print(garbage_df[display_cols].head(10).to_string())
else:
    print("\nNo garbage records detected.")


=== Sample Garbage Records (first 10) ===
      nRawStateDataID nRawStateDataID_parent txtRawStateData               txtTown approve_status ynDeleted
51359           52362                      0             NaN              Fairview        Deleted         1
51360           52363                      0             NaN      Berkeley Springs        Deleted         1
51361           52364                      0             NaN               Bethany        Deleted         1
51362           52365                      0             NaN  Blue Sulphur Springs        Deleted         1
51363           52366                      0             NaN         Capon Springs        Deleted         1
51366           52369                      0             NaN             Lexington        Deleted         1
51452           52455                      0             NaN                DELETE        Deleted         1
51453           52456                      0             NaN                DELETE        Del

In [381]:
# Progressive criteria analysis - show how records get filtered at each step

c1 = df['_is_childless']
c2 = df['_is_orphaned']
c3 = df['_empty_content']
c4 = df['_not_approved']

print(f"Starting records: {len(df):,}")
print(f"After criterion 1 (childless): {c1.sum():,}")
print(f"After criteria 1+2 (+ orphaned): {(c1 & c2).sum():,}")
print(f"After criteria 1+2+3 (+ empty content): {(c1 & c2 & c3).sum():,}")
print(f"After criteria 1+2+3+4 (+ not approved): {(c1 & c2 & c3 & c4).sum():,}")

Starting records: 52,046
After criterion 1 (childless): 8,992
After criteria 1+2 (+ orphaned): 8,788
After criteria 1+2+3 (+ empty content): 1,094
After criteria 1+2+3+4 (+ not approved): 335


---
## Transformation 4: Parent-Child Relationship Repair

The `nRawStateDataID_parent` field is used to denote hierarchical relationships where a record's field values are derived from a previous record's `txtRawStateData`. Records requiring a parent are identified by `txtRawStateData` beginning (case-insensitive) with:

- `Same` — Indicates a variant of the preceding postmark
- `(L)` — Latest known use date reference
- `*(L)` or `(L)*` — Latest known use with asterisk notation
- `*(E)` or `(E)*` — Earliest known use with asterisk notation

Since `nRawStateDataID` values were assigned in parent-then-child order during import, we can iterate through records sequentially to establish proper parentage.

### Analysis Approach

Before reassignment, we analyze the current state to identify:
1. **Type A**: Records that SHOULD have a parent (based on text indicators) but currently DON'T
2. **Type B**: Records that DO have a parent assigned but text DOESN'T indicate they should — these are anomalies requiring investigation

In [382]:
# Define parent indicator patterns
PARENT_INDICATOR_PATTERNS = [
    r'^same',       # starts with "same" (case-insensitive)
    r'^\(L\)',      # starts with "(L)"
    r'^\*\(L\)',    # starts with "*(L)"
    r'^\(L\)\*',    # starts with "(L)*"
    r'^\*\(E\)',    # starts with "*(E)"
    r'^\(E\)\*',    # starts with "(E)*"
]
COMBINED_INDICATOR_PATTERN = '|'.join(PARENT_INDICATOR_PATTERNS)

def should_have_parent(txt):
    """Check if txtRawStateData indicates this record should have a parent."""
    if pd.isna(txt) or str(txt).strip() == '':
        return False
    return bool(re.match(COMBINED_INDICATOR_PATTERN, str(txt).strip(), re.IGNORECASE))

def has_parent_assigned(row):
    """Check if nRawStateDataID_parent indicates a true parent (not self, not 0, not empty)."""
    parent = row['nRawStateDataID_parent']
    self_id = row['nRawStateDataID']
    if pd.isna(parent) or str(parent).strip() == '' or str(parent) == '0':
        return False
    return str(parent) != str(self_id)

# Apply checks to dataframe
df['_should_have_parent'] = df['txtRawStateData'].apply(should_have_parent)
df['_has_parent_assigned'] = df.apply(has_parent_assigned, axis=1)

print("=== Current Parent-Child Relationship Status ===")
print(f"Records that SHOULD have a parent (text-based): {df['_should_have_parent'].sum():,}")
print(f"Records that DO have a parent assigned: {df['_has_parent_assigned'].sum():,}")

=== Current Parent-Child Relationship Status ===
Records that SHOULD have a parent (text-based): 13,935
Records that DO have a parent assigned: 204


In [383]:
# Identify mismatches
type_a = df[df['_should_have_parent'] & ~df['_has_parent_assigned']]
type_b = df[~df['_should_have_parent'] & df['_has_parent_assigned']]

print(f"Type A (should have parent, doesn't): {len(type_a):,}")
print(f"Type B (has parent, text doesn't indicate): {len(type_b):,}")

Type A (should have parent, doesn't): 13,854
Type B (has parent, text doesn't indicate): 123


### Type B Analysis: Duplicate Records from Botched Import

Type B records are anomalies — they have a parent assigned but their `txtRawStateData` doesn't begin with any parent indicator. Investigation reveals these are **duplicate records from a failed import process**, not legitimate parent-child relationships.

In [384]:
if len(type_b) > 0:
    # Check state distribution
    print(f"\nState Distribution:")
    print(type_b['nStateID'].value_counts())
    
    # Uncomment to check creation date patterns in original data
    #print(f"\nCreation Date Distribution (top 5):")
    #print(type_b['dtEntered'].value_counts().head())
    
    # Check ID ranges
    type_b_ids = type_b['nRawStateDataID'].astype(int)
    print(f"\nID Range:")
    print(f"   Min ID: {type_b_ids.min()}")
    print(f"   Max ID: {type_b_ids.max()}")
    
    # Check if content matches parent
    print(f"\nContent Comparison with Parents:")
    identical_count = 0
    for _, row in type_b.iterrows():
        parent_id = row['nRawStateDataID_parent']
        child_txt = str(row['txtRawStateData']).strip() if pd.notna(row['txtRawStateData']) else ''
        parent_row = df[df['nRawStateDataID'] == parent_id]
        if len(parent_row) > 0:
            parent_txt = str(parent_row.iloc[0]['txtRawStateData']).strip() if pd.notna(parent_row.iloc[0]['txtRawStateData']) else ''
            if child_txt == parent_txt:
                identical_count += 1
    print(f"   Records with IDENTICAL txtRawStateData to parent: {identical_count} of {len(type_b)}")
    
    # Check approve_status
    print(f"\nApproval Status:")
    print(type_b['approve_status'].value_counts())
else:
    print("No Type B records found.")


State Distribution:
nStateID
46    90
48    33
Name: count, dtype: int64

ID Range:
   Min ID: 770
   Max ID: 973

Content Comparison with Parents:
   Records with IDENTICAL txtRawStateData to parent: 123 of 123

Approval Status:
approve_status
Approved    119
Deleted       4
Name: count, dtype: int64


In [385]:
# Display sample Type B records
if len(type_b) > 0:
    print("These records have parent assignments but text doesn't indicate parentage.")
    print("Investigation shows they are DUPLICATES with identical content to their 'parents'.\n")
    
    for _, row in type_b.head(10).iterrows():
        record_id = row['nRawStateDataID']
        parent_id = row['nRawStateDataID_parent']
        txt = str(row['txtRawStateData'])[:70] if pd.notna(row['txtRawStateData']) else 'NULL'
        print(f"ID: {record_id} | Parent: {parent_id} | Town: {row['txtTown']}")
        print(f"   txtRawStateData: {txt}...")
        print()

These records have parent assignments but text doesn't indicate parentage.
Investigation shows they are DUPLICATES with identical content to their 'parents'.

ID: 770 | Parent: 8 | Town: nan
   txtRawStateData: FREDERICKSBURG(“F” 5mm high, used as bkstp)(March 1, 1775;SL-50x3,MDD ...

ID: 771 | Parent: 8 | Town: Fredericksburg
   txtRawStateData: FREDERICKSBURG(“F” 5mm high, used as bkstp)(March 1, 1775;SL-50x3,MDD ...

ID: 777 | Parent: 115 | Town: ALDIE
   txtRawStateData: ALDIE/Va.(1851-53;33;PAID,5;Black,Blue,Brown,Red) 20...

ID: 778 | Parent: 115 | Town: ALDIE
   txtRawStateData: ALDIE/Va.(1851-53;33;PAID,5;Black,Blue,Brown,Red) 20...

ID: 779 | Parent: 115 | Town: ALDIE
   txtRawStateData: ALDIE/Va.(1851-53;33;PAID,5;Black,Blue,Brown,Red) 20...

ID: 783 | Parent: 121 | Town: Alexandria
   txtRawStateData: ALEXANDRIA/Va.(1848-54;32;5[box],5,10;Black,Red) 15...

ID: 784 | Parent: 138 | Town: AMHERST C.H
   txtRawStateData: AMHERST C.H./Va.(1838-46;30;PAID,5[C];Black,Red) 25...

ID

### Type B Diagnosis

Based on investigation, Type B records exhibit these characteristics:

| Finding | Detail |
|---------|--------|
| **States affected** | Only Virginia and West Virginia |
| **Content** | ALL have **identical** `txtRawStateData` to their assigned parents |
| **Creation date** | Most created in a single batch (April 19, 2022) |
| **Bogus dates** | Some have impossible "1905" dates |
| **ID clustering** | IDs clustered in a specific range |

**Conclusion**: These are **duplicate records from a botched import process**, not legitimate parent-child relationships. They should be:
1. Flagged as duplicates
2. Excluded from the parentage reassignment process
3. Saved separately for review/deletion

In [386]:
# Mark Type B records as duplicates
df['_is_duplicate'] = ~df['_should_have_parent'] & df['_has_parent_assigned']
duplicate_count = df['_is_duplicate'].sum()

print(f"Duplicate records identified (Type B): {duplicate_count:,}")

Duplicate records identified (Type B): 123


### Parent-Child Reassignment

Now we perform the actual parentage reassignment for Type A records (those that should have a parent but don't). The algorithm:

1. Sort records by `nRawStateDataID` (ascending) to process in import order
2. Track the current "parent context" as we iterate
3. When a record's text begins with a parent indicator, assign it the current parent
4. When a record's text does NOT begin with a parent indicator, it becomes the new parent context
5. Skip duplicate records (Type B) — they retain their existing (incorrect) parent assignment for now

In [387]:
def reassign_parents(dataframe):
    """
    Reassign nRawStateDataID_parent based on txtRawStateData indicators.
    
    Returns a copy of the dataframe with updated parent assignments and 
    a count of changes made.
    """
    df_work = dataframe.copy()
    
    # Sort by ID to ensure proper ordering
    df_work = df_work.sort_values('nRawStateDataID', key=lambda x: x.astype(int))
    df_work = df_work.reset_index(drop=True)
    
    # Track changes
    changes = 0
    
    # Current parent context (the most recent "root" record)
    current_parent_id = None
    
    # Store new parent values
    new_parents = []
    
    for idx, row in df_work.iterrows():
        record_id = row['nRawStateDataID']
        old_parent = row['nRawStateDataID_parent']
        txt = str(row['txtRawStateData']).strip() if pd.notna(row['txtRawStateData']) else ''
        is_duplicate = row['_is_duplicate'] if '_is_duplicate' in row else False
        
        # Skip duplicates - leave their parent as-is
        if is_duplicate:
            new_parents.append(old_parent)
            continue
        
        # Check if this record should have a parent
        needs_parent = should_have_parent(txt)
        
        if needs_parent and current_parent_id is not None:
            # This is a child record - assign current parent
            new_parent = current_parent_id
            if str(old_parent) != str(new_parent):
                changes += 1
            new_parents.append(new_parent)
        else:
            # This is a root record - becomes new parent context
            current_parent_id = record_id
            # Root records are their own parent (self-referencing)
            new_parent = record_id
            if str(old_parent) != str(new_parent):
                changes += 1
            new_parents.append(new_parent)
    
    # Apply new parent values
    df_work['nRawStateDataID_parent'] = new_parents
    
    return df_work, changes

print(f"Records to process: {len(df):,}")
print(f"Duplicates to skip: {duplicate_count:,}")

Records to process: 52,046
Duplicates to skip: 123


In [388]:
# Perform the reassignment
df, parent_changes = reassign_parents(df)

print(f"Total parent values changed: {parent_changes:,}")

Total parent values changed: 19,039


In [389]:
# Verify the reassignment
df['_should_have_parent'] = df['txtRawStateData'].apply(should_have_parent)
df['_has_parent_assigned'] = df.apply(has_parent_assigned, axis=1)

# Exclude duplicates from verification
df_non_dup = df[~df['_is_duplicate']]

type_a_after = df_non_dup[df_non_dup['_should_have_parent'] & ~df_non_dup['_has_parent_assigned']]

print(f"Records that SHOULD have parent: {df_non_dup['_should_have_parent'].sum():,}")
print(f"Records that DO have parent: {df_non_dup['_has_parent_assigned'].sum():,}")
print(f"Remaining Type A mismatches: {len(type_a_after):,}")

if len(type_a_after) > 0:
    print(f"\nNote: {len(type_a_after)} records still lack parents. These may be:")
    print("  - First record in a sequence (no prior parent available)")
    print("  - Records with indicators but no valid parent context")

Records that SHOULD have parent: 13,935
Records that DO have parent: 13,935
Remaining Type A mismatches: 0


In [390]:
# Show sample of reassigned relationships
children = df[(df['_should_have_parent']) & (df['_has_parent_assigned']) & (~df['_is_duplicate'])]

for _, child in children.head(10).iterrows():
    child_id = child['nRawStateDataID']
    parent_id = child['nRawStateDataID_parent']
    child_txt = str(child['txtRawStateData'])[:60] if pd.notna(child['txtRawStateData']) else 'NULL'
    
    # Get parent record
    parent = df[df['nRawStateDataID'] == parent_id]
    if len(parent) > 0:
        parent_txt = str(parent.iloc[0]['txtRawStateData'])[:60] if pd.notna(parent.iloc[0]['txtRawStateData']) else 'NULL'
        print(f"Parent {parent_id}: {parent_txt}...")
        print(f"  └─ Child {child_id}: {child_txt}...")
        print()

Parent 1: Alexa.(Alexandria)(E)(May 21, 1772;Ms;Black) 1,500...
  └─ Child 2: (L)(Sept. 15, 1774) 1,000...

Parent 8: FREDERICKSBURG(“F” 5mm high, used as bkstp)(March 1, 1775;SL...
  └─ Child 9: (L)(June 27, 1775) 1,000...

Parent 15: Norf(Norfolk)(E)(Aug. 21, 1765;Ms;Black) 1,500...
  └─ Child 16: (L)(Dec. 6, 1765) 1,200...

Parent 17: Nfk(Norfolk)(Nov. 20, 1772;Ms;Red) 1,000...
  └─ Child 18: Same(--, 1773;Ms;Black) 1,000...

Parent 19: NORFOLK(backstamp)(E)(Feb. 11, 1775;SL-29x5,MDD below;Black)...
  └─ Child 20: (L)(Oct. 2, 1775) 1,200...

Parent 19: NORFOLK(backstamp)(E)(Feb. 11, 1775;SL-29x5,MDD below;Black)...
  └─ Child 21: Same(May 6, 1775;MDD same line) 1,500...

Parent 19: NORFOLK(backstamp)(E)(Feb. 11, 1775;SL-29x5,MDD below;Black)...
  └─ Child 22: Same(July 25, 1775;MDD below;Red) 1,500...

Parent 26: SUFFOLK(April 12, 1775;SL-30x5,MDD below;Red) 1500...
  └─ Child 27: Same(July 25, 1775;Black) 1,200...

Parent 28: WBurg(Williamsburg)(E)(Nov. 14, 1734;Ms;Black) 2,500...


---
## Transformation 5: Family Text Rollup (txtRawStateDataTemp)

After parent-child relationships are established, we create a "rolled up" view of each catalog entry family. The `txtRawStateDataTemp` field is populated with an **uppercase concatenation** of:

1. The parent record's `txtRawStateData`
2. All child records' `txtRawStateData` (sorted by `nRawStateDataID`)

This provides a complete, searchable view of the entire catalog entry including all variants.

### Example
**Before rollup:**
- Parent (ID 1): `Alexa.(Alexandria)(E)(May 21, 1772;Ms;Black) 1,500`
- Child (ID 2): `(L)(Sept. 15, 1774) 1,000`

**After rollup (txtRawStateDataTemp for both records):**
`ALEXA.(ALEXANDRIA)(E)(MAY 21, 1772;MS;BLACK) 1,500  (L)(SEPT. 15, 1774) 1,000`

Records without children (standalone entries) receive their own `txtRawStateData` uppercased.

In [391]:
def create_family_rollup(dataframe, delimiter=' | '):
    df = dataframe.copy()

    # Pass 1: clean text (only from the source column)
    df['_txt_clean'] = (
        df['txtRawStateData']
          .fillna('')
          .astype(str)
          .replace(['nan', 'None', 'null', 'NaN'], '')
          .str.strip()
    )

    # Build stable numeric IDs (avoids '1' vs '1.0' vs 'nan' string issues)
    id_num = pd.to_numeric(df['nRawStateDataID'], errors='coerce')
    parent_num = pd.to_numeric(df['nRawStateDataID_parent'], errors='coerce')

    # Stable family id:
    # - if parent exists => family is that parent id
    # - else => family is the row's own id (root)
    df['_family_id'] = parent_num.where(parent_num.notna(), id_num)

    # If both ID and parent are missing, isolate by row index
    # IMPORTANT: must be a Series, not an Index
    fallback = df.index.to_series().astype('int64')
    df['_family_id'] = df['_family_id'].fillna(fallback)

    # Parent-first ordering inside each family:
    # parent row is the one whose nRawStateDataID == _family_id
    df['_is_parent'] = (id_num == df['_family_id']).fillna(False)

    # Sort so parent is first, then by ID for stable ordering
    df_sorted = df.sort_values(
        by=['_family_id', '_is_parent', 'nRawStateDataID'],
        ascending=[True, False, True]
    )

    # Pass 2: compute one rollup per family, then broadcast via map
    def rollup_text(series):
        vals = [v for v in series if v]
        if not vals:
            return ""

        # If you do NOT want de-duping, remove this block.
        seen = set()
        uniq = []
        for v in vals:
            if v not in seen:
                seen.add(v)
                uniq.append(v)

        return delimiter.join(uniq).upper()

    rollups = (
        df_sorted
        .groupby('_family_id', sort=False)['_txt_clean']
        .apply(rollup_text)
    )

    df['txtRawStateDataTemp'] = df['_family_id'].map(rollups)

    # Metrics
    valid = df['txtRawStateDataTemp'] != ""
    family_count = df.loc[valid, '_family_id'].nunique()
    record_count = int(valid.sum())

    print("Family rollup complete:")
    print(f"  Families processed: {family_count:,}")
    print(f"  Records updated: {record_count:,}")

    return df


In [392]:
# Perform the family text rollup
df = create_family_rollup(df, delimiter='  ')

Family rollup complete:
  Families processed: 36,872
  Records updated: 50,930


### Statistics on rollup

In [393]:
# Check txtRawStateDataTemp population
temp_populated = df['txtRawStateDataTemp'].notna() & (df['txtRawStateDataTemp'] != '')
print(f"Records with txtRawStateDataTemp populated: {temp_populated.sum():,} ({temp_populated.sum()/len(df)*100:.1f}%)")

# Count families by size
family_sizes = df.groupby('nRawStateDataID_parent').size()
print(f"\nFamily size distribution:")
print(f"  Single records (no children): {(family_sizes == 1).sum():,}")
print(f"  2 records (parent + 1 child): {(family_sizes == 2).sum():,}")
print(f"  3+ records: {(family_sizes >= 3).sum():,}")
print(f"  Largest family: {family_sizes.max()} records")

Records with txtRawStateDataTemp populated: 50,930 (97.9%)

Family size distribution:
  Single records (no children): 30,793
  2 records (parent + 1 child): 4,440
  3+ records: 2,755
  Largest family: 82 records


---
## Save Results

In [394]:
# Separate records by category
# Duplicates: Type B records from botched import (special case)
duplicate_records = df[df['_is_duplicate']]

# Clean: Approved AND not deleted AND not duplicate
clean_mask = (
    (df['approve_status'] == 'Approved') & 
    (~df['_is_duplicate'])
)
clean_records = df[clean_mask]

# Garbage: Everything else (not clean AND not duplicate)
garbage_records = df[~clean_mask & ~df['_is_duplicate']]

# Uncomment to drop temporary/work fields for final output
#"""
temp_cols = [ '_is_childless',
              '_is_orphaned',
              '_empty_content',
              '_not_approved',
              '_should_have_parent',
              '_has_parent_assigned',
              '_is_duplicate',
              '_is_garbage',
              '_txt_clean',
              '_family_id',
              '_is_parent',
              'txtRawStateDataTemp',
              'approve_status',
              'ynDeleted' ]
clean_records = clean_records.drop(columns=temp_cols)
duplicate_records = duplicate_records.drop(columns=temp_cols)
garbage_records = garbage_records.drop(columns=temp_cols)
#"""

# Ensure output directory exists
Path(OUTPUT_FILE).parent.mkdir(parents=True, exist_ok=True)

# Save cleaned data
clean_records.to_csv(OUTPUT_FILE, index=False)
print(f"Saved {len(clean_records):,} clean records to: {OUTPUT_FILE}")

# Save garbage records separately for review
garbage_records.to_csv(GARBAGE_FILE, index=False)
print(f"Saved {len(garbage_records):,} garbage records to: {GARBAGE_FILE}")

# Save duplicate records separately for review
duplicate_records.to_csv(DUPLICATES_FILE, index=False)
print(f"Saved {len(duplicate_records):,} duplicate records to: {DUPLICATES_FILE}")

Saved 51,392 clean records to: ./wip/out/tblRawStateData.csv
Saved 531 garbage records to: ./wip/out/tblRawStateData_garbage.csv
Saved 123 duplicate records to: ./wip/out/tblRawStateData_duplicates.csv
